# Ungraded Lab - Tracing a RAG system

---

Welcome to the ungraded lab on tracing and evaluating a RAG system using Weaviate and Phoenix! In this interactive session, you will learn how to effectively use telemetry to trace and troubleshoot a RAG system. You'll get to understand and work with key concepts, including spans, traces, and chains, which are essential in monitoring and improving system performance.

In this ungraded lab, you will:

- Understand how to set up and use telemetry to monitor your RAG system.
- Learn about traces and spans.
- Explore traces to see the complete path and interactions within your system processes.
- Use the [Phoenix Arize](https://phoenix.arize.com/) tool for visualizing and analyzing telemetry data.
- See a small RAG pipeline in action using Phoenix and Weaviate.
  

---

<h4 style="color:black; font-weight:bold;">USING THE TABLE OF CONTENTS</h4>
JupyterLab provides an easy way for you to navigate through your assignment. It's located under the Table of Contents tab, found in the left panel, as shown in the picture below.

![TOC Location](images/toc.png)

---


# Table of Contents
- [ 1 - Introduction](#1)
  - [ 1.1 Importing necessary libraries](#1-1)
- [ 2 - Quick Introduction on Telemetry](#2)
  - [ 2.1 Spans](#2-1)
- [ 3 - Telemetry Using Phoenix](#3)
  - [ 3.1 Launching Phoenix App](#3-1)
  - [ 3.2 Preparing the telemetry](#3-2)
  - [ 3.3 Working the Pipeline](#3-3)
  - [ 3.4 Chains](#3-4)
  - [ 3.5 Using the UI to analyze the traces](#3-5)
- [ 4 - Tracing and Evaluation with Weaviate](#4)
  - [ 4.1 Configuring the tracer](#4-1)
  - [ 4.2 Preparing the Weaviate client and collection](#4-2)
  - [ 4.3 The Retriever](#4-3)
  - [ 4.4 LLM call with `openai` library](#4-4)
- [ 5 - Evaluating a RAG system](#5)


<a id='1'></a>
## 1 - Introduction

---
In the context of RAG systems, telemetry is key for monitoring and optimizing performance. By collecting and transmitting data on the system's operations, such as spans (individual steps) and traces (full workflows), telemetry provides a way to watch how the system retrieves, processes, and generates information. This visibility helps identify bottlenecks and diagnose issues, improving system efficiency.

<a id='1-1'></a>
### 1.1 Importing necessary libraries


In [1]:
import utils
from opentelemetry import trace
from opentelemetry.sdk.resources import Resource
from opentelemetry.trace import Status, StatusCode
from opentelemetry.sdk.trace import TracerProvider
from opentelemetry.sdk.trace.export import ConsoleSpanExporter, SimpleSpanProcessor

<a id='2'></a>
## 2 - Quick Introduction on Telemetry
---
<a id='2-1'></a>
### 2.1 Spans

In telemetry, a span represents a single operation or task within your system. It's like a snapshot of a specific action, recording when it starts and ends. Spans also include details like what the task is doing and any important events that occur. By tracking spans, you can see how long operations take and spot any issues, helping you understand and improve your system's performance.

Let's see an example of how to setup a simple tracer using [OpenTelemetry](https://opentelemetry.io/) - this tool is also used by Phoenix.

Note an error reading "Overriding of current TracerProvider is not allowed" is generated by the following cell but will not affect the functionality of this lab.

In [2]:
# Define a resource with attributes that describe your application
# Here, we're setting the service name to identify what is being traced
resource = Resource(attributes={
    "service.name": "Test Service"
})

# Set up the tracer provider that will manage and provide tracers
# 'TracerProvider' is initialized with the resource we just defined
trace.set_tracer_provider(TracerProvider(resource=resource))

# Create a console exporter to output spans to the console for demonstration purposes
# In a real-world scenario, you might use an OTLP exporter to send spans to a tracing system
console_exporter = ConsoleSpanExporter()

# Set up a span processor to handle the spans
# SimpleSpanProcessor sends each span to the exporter as soon as it is finished
span_processor = SimpleSpanProcessor(console_exporter)

# Add the span processor to the tracer provider to start processing spans immediately
trace.get_tracer_provider().add_span_processor(span_processor)

# Obtain a tracer for the current module to create and manage spans
tracer = trace.get_tracer(__name__)

#### 2.1.1 A Toy Retrieve Function

This is a basic function designed to illustrate how to set up tracing using spans for a document retrieval operation.

In [3]:
def retrieve(query, fail=False):
    # Start a span to trace the retrieval process
    with tracer.start_as_current_span("retrieving_documents") as span:
        # Log the event of starting retrieval
        span.add_event("Starting retrieve")
        # Record the input query as an attribute for visibility
        span.set_attribute("input.query", query)
        try:
            # Simulate a retrieval failure if 'fail' is True
            if fail:
                raise ValueError(f"Retrieve failed for query: {query}")

            # Simulated list of retrieved documents
            retrieved_docs = ['retrieved doc1', 'retrieved doc2', 'retrieved doc3']
            # Record details about each retrieved document
            for i, doc in enumerate(retrieved_docs):
                span.set_attribute(f"retrieval.documents.{i}.document.id", i)
                span.set_attribute(f"retrieval.documents.{i}.document.content", doc)
                span.set_attribute(f"retrieval.documents.{i}.document.metadata", f"Metadata for document {i}")
        except Exception as e:
            # If an exception occurs, log and set the span status to indicate an error
            span.set_status(Status(StatusCode.ERROR, str(e)))
            span.set_attribute("error.type", type(e).__name__)
            span.set_attribute("error.message", str(e))
            # Reraise the exception for handling by the caller
            raise

        # Mark the span as successful if no error was raised
        span.set_status(Status(StatusCode.OK))
        return retrieved_docs

In [4]:
# The tracer is configured to show the span in the output
retrieve("Test")

{
    "name": "retrieving_documents",
    "context": {
        "trace_id": "0x1f36ddc8835f4ab3ce9a1330dd94999f",
        "span_id": "0x00928798b6539174",
        "trace_state": "[]"
    },
    "kind": "SpanKind.INTERNAL",
    "parent_id": null,
    "start_time": "2026-02-08T22:14:24.347631Z",
    "end_time": "2026-02-08T22:14:24.347718Z",
    "status": {
        "status_code": "OK"
    },
    "attributes": {
        "input.query": "Test",
        "retrieval.documents.0.document.id": 0,
        "retrieval.documents.0.document.content": "retrieved doc1",
        "retrieval.documents.0.document.metadata": "Metadata for document 0",
        "retrieval.documents.1.document.id": 1,
        "retrieval.documents.1.document.content": "retrieved doc2",
        "retrieval.documents.1.document.metadata": "Metadata for document 1",
        "retrieval.documents.2.document.id": 2,
        "retrieval.documents.2.document.content": "retrieved doc3",
        "retrieval.documents.2.document.metadata": "M

['retrieved doc1', 'retrieved doc2', 'retrieved doc3']

## 2.2 Traces
---
A trace is a collection of spans that represent the journey of a request or transaction as it moves through various components in your system. It is a set of spans that are related to one task.

Now let's complete a toy rag pipeline to see what a trace would look like.

In [5]:
def format_documents(retrieved_docs):
    # Start a span to trace the formatting of documents
    with tracer.start_as_current_span("call_format_documents") as span:
        # Log the event for initiating document formatting
        span.add_event("Calling format_documents")
        # Record the number of documents being formatted
        span.set_attribute("input.documents_count", len(retrieved_docs))

        t = ''
        for i, doc in enumerate(retrieved_docs):
            t += f'Retrieved doc: {doc}\n'
            # Log an event for each processed document
            span.add_event(f"processed document {i}", {"document.content": doc})

        # Mark the span as successful after formatting documents
        span.set_status(Status(StatusCode.OK))
    return t

def augment_prompt(query, formatted_documents):
    # Start a span to trace the prompt augmentation process
    with tracer.start_as_current_span("augment_prompt") as span:
        # Log the event for the beginning of prompt augmentation
        span.add_event("Starting prompt augmentation")
        # Record input details such as the query and document length
        span.set_attribute("input.query", query)
        span.set_attribute("input.formatted_documents_length", len(formatted_documents))

        # Create a prompt that combines the query and formatted documents
        PROMPT = f"Answer the query: {query}.\nRelevant documents:\n{formatted_documents}"

        # Mark the span as successful
        span.set_status(Status(StatusCode.OK))
    return PROMPT

def generate(prompt):
    # Start a span to trace the text generation based on the prompt
    with tracer.start_as_current_span("generate") as span:
        # Log the event for starting text generation
        span.add_event("Starting text generation")
        # Record the prompt being used for generation
        span.set_attribute("input.prompt", prompt)

        # Simulate the text generation process
        generated_text = f"Generated text for prompt {prompt}"

        # Mark the span as successful after text generation
        span.set_status(Status(StatusCode.OK))
    return generated_text

def rag_pipeline(query, fail = False):
    # Start a span to trace the entire RAG pipeline process
    with tracer.start_as_current_span("rag_pipeline") as span:
        try:
            # Step 1: Retrieve documents based on the query
            retrieved_docs = retrieve(query, fail = fail)
            # Step 2: Format the retrieved documents
            formatted_docs = format_documents(retrieved_docs)
            # Step 3: Augment the query with relevant documents to form a prompt
            prompt = augment_prompt(query, formatted_docs)
            # Step 4: Generate a response from the augmented prompt
            generated_response = generate(prompt)

            # Mark the span as successful when all steps are completed
            span.set_status(Status(StatusCode.OK))
            return generated_response
        except Exception as e:
            # If any step raises an exception, set the span status to error
            span.set_status(Status(StatusCode.ERROR, str(e)))
            # Reraise the exception for external handling
            raise

In [6]:
# Trace example 1
response = rag_pipeline("This is a test query", fail = False)

{
    "name": "retrieving_documents",
    "context": {
        "trace_id": "0xe6922f53f8a0218dd2c9d9482d66528c",
        "span_id": "0x9b0e8d9b67101ba4",
        "trace_state": "[]"
    },
    "kind": "SpanKind.INTERNAL",
    "parent_id": "0x4b429845e775e640",
    "start_time": "2026-02-08T22:18:18.738897Z",
    "end_time": "2026-02-08T22:18:18.738963Z",
    "status": {
        "status_code": "OK"
    },
    "attributes": {
        "input.query": "This is a test query",
        "retrieval.documents.0.document.id": 0,
        "retrieval.documents.0.document.content": "retrieved doc1",
        "retrieval.documents.0.document.metadata": "Metadata for document 0",
        "retrieval.documents.1.document.id": 1,
        "retrieval.documents.1.document.content": "retrieved doc2",
        "retrieval.documents.1.document.metadata": "Metadata for document 1",
        "retrieval.documents.2.document.id": 2,
        "retrieval.documents.2.document.content": "retrieved doc3",
        "retrieval.do

In [7]:
# Trace example 2
response = rag_pipeline("This is a test query", fail = True)

{
    "name": "retrieving_documents",
    "context": {
        "trace_id": "0x76a90645e2ea26dc38bcea3586d54656",
        "span_id": "0x8f0de246d860780d",
        "trace_state": "[]"
    },
    "kind": "SpanKind.INTERNAL",
    "parent_id": "0x8dcc6ad16553b2bb",
    "start_time": "2026-02-08T22:19:48.810306Z",
    "end_time": "2026-02-08T22:19:48.813210Z",
    "status": {
        "status_code": "ERROR",
        "description": "ValueError: Retrieve failed for query: This is a test query"
    },
    "attributes": {
        "input.query": "This is a test query",
        "error.type": "ValueError",
        "error.message": "Retrieve failed for query: This is a test query"
    },
    "events": [
        {
            "name": "Starting retrieve",
            "timestamp": "2026-02-08T22:19:48.810319Z",
            "attributes": {}
        },
        {
            "name": "exception",
            "timestamp": "2026-02-08T22:19:48.813189Z",
            "attributes": {
                "exception.t

ValueError: Retrieve failed for query: This is a test query

Note: This second trace intentionally failed to show what that might look like in a producton system.

As you can see, traces can become quite complex and difficult to read in their raw form, especially in large systems with many interconnected components. This is why tools like Phoenix are important. They help manage and visualize traces, making it easier to analyze the data and diagnose performance issues or bottlenecks efficiently.

In [ ]:
# Call the function to restart the kernel - necessary as it is not possible to overwrite a tracer in Jupyter Notebook
utils.restart_kernel()

<a id='3'></a>
## 3 - Telemetry Using Phoenix
---

Phoenix is a powerful tool designed to simplify the management and visualization of telemetry data. It helps you handle complex traces, making it easier to analyze and diagnose issues in your system. With Phoenix, you can monitor your RAG system's performance, identify bottlenecks, and gain insights into how different components of your application interact. In this section, you'll explore how to set up Phoenix and leverage its features to better understand the telemetry data generated by your RAG system.

In [1]:
import utils
import phoenix as px

<a id='3-1'></a>
### 3.1 Launching Phoenix App

Run the next cell to launch the Phoenix app, which will set up a local server and host a user interface (UI). The default URL for accessing the app is `localhost:6006`, and this will be displayed once you call the application. However, due to the limitations of the Coursera environment, a different link will be provided. You can click this alternate link to navigate to the UI in a new tab.

In [3]:
utils.make_url()
px.launch_app()

Existing running Phoenix instance detected! Shutting it down and starting a new instance...


FOLLOW THIS URL TO OPEN THE UI: http://iburgqpyevpe.labs.coursera.org
🌍 To view the Phoenix app in your browser, visit http://localhost:6006/
📖 For more information on how to use Phoenix, check out https://arize.com/docs/phoenix


You should see something like this:

![Phoenix UI Screenshot](images/ui_1.png)

<a id='3-2'></a>
### 3.2 Preparing the telemetry

Now you'll configure the telemetry to work with Phoenix. Since Phoenix also uses OpenTelemetry, the setup is very similar to what you've seen above.

In [4]:
from phoenix.otel import register
from opentelemetry.trace import Status, StatusCode
phoenix_project_name = "example-rag-pipeline"

# With phoenix, we just need to register to get the tracer provider with the appropriate endpoint.
endpoint="http://127.0.0.1:6006/v1/traces"
tracer_provider_phoenix = register(project_name=phoenix_project_name, endpoint = endpoint)

# Retrieve a tracer for manual instrumentation
tracer = tracer_provider_phoenix.get_tracer(__name__)

🔭 OpenTelemetry Tracing Details 🔭
|  Phoenix Project: example-rag-pipeline
|  Span Processor: SimpleSpanProcessor
|  Collector Endpoint: http://127.0.0.1:6006/v1/traces
|  Transport: HTTP + protobuf
|  Transport Headers: {}
|  
|  Using a default SpanProcessor. `add_span_processor` will overwrite this default.
|  
|  ⚠️ WARNING: It is strongly advised to use a BatchSpanProcessor in production environments.
|  
|  `register` has set this TracerProvider as the global OpenTelemetry default.
|  To disable this behavior, call `register` with `set_global_tracer_provider=False`.



<a id='3-3'></a>
### 3.3 Using the Pipeline

#### 3.3.1 Retrieve

This is the same toy retrieve function. The syntax is almost the same, but there are two differences:

1. Now there is a `openinference_span_kind`, where you can pass this as a retriever.
2. You can now set the input using `span.set_input`. 

In [5]:
def retrieve(query, fail=False):
    # Start a span to trace the retrieval process. Now we can pass a span kind: retriever
    with tracer.start_as_current_span("retrieving_documents", openinference_span_kind = 'retriever') as span:
        # Log the event of starting retrieval
        span.add_event("Starting retrieve")
        # Record the input query as an attribute for visibility
        # Phoenix allows you to use span.set_input
        span.set_input(query)
        try:
            # Simulate a retrieval failure if 'fail' is True
            if fail:
                raise ValueError(f"Retrieve failed for query: {query}")

            # Simulated list of retrieved documents
            retrieved_docs = ['retrieved doc1', 'retrieved doc2', 'retrieved doc3']
            # Record details about each retrieved document
            for i, doc in enumerate(retrieved_docs):
                span.set_attribute(f"retrieval.documents.{i}.document.id", i)
                span.set_attribute(f"retrieval.documents.{i}.document.content", doc)
                span.set_attribute(f"retrieval.documents.{i}.document.metadata", f"Metadata for document {i}")
        except Exception as e:
            # If an exception occurs, log and set the span status to indicate an error
            span.set_status(Status(StatusCode.ERROR, str(e)))
            span.set_attribute("error.type", type(e).__name__)
            span.set_attribute("error.message", str(e))
            # Reraise the exception for handling by the caller
            raise

        # Mark the span as successful if no error was raised
        span.set_status(Status(StatusCode.OK))
        return retrieved_docs

<a id='3-4'></a>
### 3.4 Chains

A chain is a connection point between different steps in an LLM application. It links together various operations, like starting a request or passing information from a retriever to an LLM call. Chains help keep things organized and simple. 

#### 3.4.1 The remaining RAG functions

These are the same functions you worked before, but now with the *decorator* `@tracer.chain`. You just need to add it before the function you want to be traced and it will be added as a chain! If you want to make a more detailed tracing, then you should proceed as the retriever above.

In [6]:
@tracer.chain
def format_documents(retrieved_docs):
    t = ''
    for i, doc in enumerate(retrieved_docs):
        t += f'Retrieved doc: {doc}\n'
    return t

@tracer.chain
def augment_prompt(query, formatted_documents):
    
    # Create a prompt that combines the query and formatted documents
    PROMPT = f"Answer the query: {query}.\nRelevant documents:\n{formatted_documents}"
    return PROMPT

@tracer.chain
def generate(prompt):
    generated_text = f"Generated text for prompt {prompt}"
    return generated_text

@tracer.chain
def rag_pipeline(query, fail = False):
        # Step 1: Retrieve documents based on the query
        retrieved_docs = retrieve(query, fail = fail)
        # Step 2: Format the retrieved documents
        formatted_docs = format_documents(retrieved_docs)
        # Step 3: Augment the query with relevant documents to form a prompt
        prompt = augment_prompt(query, formatted_docs)
        # Step 4: Generate a response from the augmented prompt
        generated_response = generate(prompt)
        return generated_response

<a id='3-5'></a>
### 3.5 Using the UI to analyze the traces

Now let's get to the fun part! Run the following cell to run the same two queries as before. Then let's go to the Phoenix UI to inspect how it is placed there!

In [7]:
response = rag_pipeline("This is a test query")
try:
    response = rag_pipeline("This is a test query that failed", fail = True)
except:
    pass

In [8]:
utils.make_url()

FOLLOW THIS URL TO OPEN THE UI: http://iburgqpyevpe.labs.coursera.org


You will see something like this, and all the information is shown to you in an organized way.


![Phoenix UI Screenshot](images/ui_3.png)

In [ ]:
# Let's restart the kernel to work with a more complex example!
utils.restart_kernel()

<a id='4'></a>
## 4 - Tracing and Evaluation with Weaviate

---

Now you are familiar with the basics of telemetry with Phoenix, let's work on a more concrete scenario. Let's get our FAQ questions from M4 Assignment and implement a small RAG pipeline to answer an FAQ related question for a clothing store.

In [1]:
from phoenix.otel import register
from opentelemetry.trace import Status, StatusCode
import phoenix as px
import flask_app
import weaviate
import utils
import weaviate_server

 * Serving Flask app 'flask_app'
 * Debug mode: off


In [2]:
utils.make_url()
session = px.launch_app()

FOLLOW THIS URL TO OPEN THE UI: http://iburgqpyevpe.labs.coursera.org
🌍 To view the Phoenix app in your browser, visit http://localhost:6006/
📖 For more information on how to use Phoenix, check out https://arize.com/docs/phoenix


<a id='4-1'></a>
### 4.1 Configuring the tracer

The setup is the same as before, but now there is a new argument called `auto_instrument`. Passing it as `True` will automatically trace OpenAI compatible LLM calls!

In [3]:
from phoenix.otel import register
phoenix_project_name = "example-rag-pipeline-with-weaviate"

# With phoenix, we just need to register to get the tracer provider with the appropriate endpoint. Providing auto_instrument = True, OpenAI calls are automatically traced
# TogetherAI is OpenAI compatible!
tracer_provider_phoenix = register(project_name=phoenix_project_name, endpoint="http://127.0.0.1:6006/v1/traces", auto_instrument=True)

# Retrieve a tracer for manual instrumentation
tracer = tracer_provider_phoenix.get_tracer(__name__)

🔭 OpenTelemetry Tracing Details 🔭
|  Phoenix Project: example-rag-pipeline-with-weaviate
|  Span Processor: SimpleSpanProcessor
|  Collector Endpoint: http://127.0.0.1:6006/v1/traces
|  Transport: HTTP + protobuf
|  Transport Headers: {}
|  
|  Using a default SpanProcessor. `add_span_processor` will overwrite this default.
|  
|  ⚠️ WARNING: It is strongly advised to use a BatchSpanProcessor in production environments.
|  
|  `register` has set this TracerProvider as the global OpenTelemetry default.
|  To disable this behavior, call `register` with `set_global_tracer_provider=False`.



<a id='4-2'></a>
### 4.2 Preparing the Weaviate client and collection


In [4]:
# Connecting the weaviate client
client = weaviate.connect_to_local(port=8079, grpc_port=50050)

In [5]:
import joblib
data = joblib.load("faq.joblib")

In [6]:
# Let's recall the data structure
data[0]

{'question': 'What are your store hours?',
 'answer': 'Our online store is open 24/7. Customer service is available from 9:00 AM to 6:00 PM, Monday through Friday.',
 'type': 'general information'}

In [7]:
# Loading the collection
collection = client.collections.get("Faq")

In [8]:
len(collection)

26

<a id='4-3'></a>
### 4.3 The Retriever

Now you'll set up a retriever like you've done before. This time, you'll also add telemetry to track and understand the retrieval process. 

In [9]:
def retrieve(query_text, limit=5):
    # Start a span for the query
    with tracer.start_as_current_span(
        "query_weaviate", openinference_span_kind="retriever"
    ) as span:
        # Set the input for the span
        span.set_input(query_text)

        # Query the collection
        collection_name = "Faq"
        chunks = client.collections.get(collection_name)
        results = chunks.query.near_text(query=query_text, limit=limit)

        # Set the retrieved documents as attributes on the span
        for i, document in enumerate(results.objects):
            span.set_attribute(f"retrieval.documents.{i}.document.id", str(document.uuid))
            span.set_attribute(f"retrieval.documents.{i}.document.metadata", str(document.metadata))
            span.set_attribute(
                f"retrieval.documents.{i}.document.content", str(document.properties)
            )  

        return results

In [10]:
# Process and format the retrieved results
@tracer.chain 
def format_context(results):
    context = ""
    for item in results.objects:
        properties = item.properties
        context += f"Question: {properties['question']}\n"
        context += f"Answer: {properties['answer']}\n"
    return context
     

In [11]:
# Create a prompt with the retrieved information
@tracer.chain
def create_prompt(query_text, context):
    prompt = f"""
Based on the following information, please answer the FAQ related question: "{query_text}"

Relevant FAQ (ordered by relevance):
{context}
"""
    return prompt

<a id='4-4'></a>
### 4.4 LLM call with `openai` library

Since Phoenix integrates with OpenAI-like systems, let's use it. Luckily [together.ai](https://www.together.ai/) is OpenAI compatible! 

In [12]:
import httpx
from openai import OpenAI, DefaultHttpxClient

In [13]:
# Custom transport to bypass SSL verification
transport = httpx.HTTPTransport(local_address="0.0.0.0", verify=False)

# Create a DefaultHttpxClient instance with the custom transport
http_client = DefaultHttpxClient(transport=transport)

# You can use any openai compatible endpoint here!
llm_client = OpenAI(
    api_key = '', # Set any as the proxy running here does not use it. Set the together api key if using the together endpoint
    base_url="http://proxy.dlai.link/coursera_proxy/together/", # If using together endpoint, add it here https://api.together.xyz/
   http_client=http_client, # ssl bypass to make it work via proxy calls, remove it if running with together.ai endpoint 
)

In [14]:
# There is no need to trace as the auto_instrument was set to true
def query_openai(prompt):
    response = llm_client.chat.completions.create(
        model="meta-llama/Llama-3.2-3B-Instruct-Turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant from a customer support."},
            {"role": "user", "content": prompt},
        ],
    )
    return response.choices[0].message.content

In [15]:
@tracer.chain
def rag_pipeline(query):
    # Execute the query
    retrieved_documents = retrieve(query)
    context = format_context(retrieved_documents)
    
    # Create a prompt with the retrieved information
    final_prompt = create_prompt(query, context)
    
    # Execute the OpenAI query
    final_answer = query_openai(final_prompt)

    return final_answer

In [16]:
response = rag_pipeline("Can I get a refund or exchange for another shirt?")
print(response)

Based on the provided information, here's the answer to your question:

"Unfortunately, you cannot get a refund or exchange for another shirt, as sale items are final sale and cannot be returned or exchanged, unless stated otherwise. However, if you purchased the shirt within the 30-day return policy timeframe and it's not a sale item, you may be able to initiate an exchange through our Returns Center."


In [17]:
response = rag_pipeline("What are your working hours?")
print(response)

Based on the provided information, the answer to the FAQ question "What are your working hours?" is:

Our customer service is available from 9:00 AM to 6:00 PM, Monday through Friday.


In [18]:
# Checkout the traces in the Phoenix UI!
utils.make_url()

FOLLOW THIS URL TO OPEN THE UI: http://iburgqpyevpe.labs.coursera.org


Keep it up! You finished the ungraded lab on Telemetry with Phoenix!